In [24]:
import qiskit

from PatchedMeasCal.tensor_patch_cal import TensorPatchFitter
from PatchedMeasCal.fake_backends import Grid, Hexagonal13, Hexagonal16, FullyConnected, SquareOctagonal, Hexagonal
from PatchedMeasCal.inv_measure_methods import aim, sim
from PatchedMeasCal import jigsaw

from PatchedMeasCal import state_prep_circuits
from PatchedMeasCal.fake_measurement_distributions import renormalise_measurement_results
from PatchedMeasCal.utils import Progressbar
from PatchedMeasCal.gen_error_matrices import FakeMeasurementError

from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
from qiskit.ignis.mitigation.measurement import tensored_meas_cal, TensoredMeasFitter
from PatchedMeasCal.qiskit_meas_fitters import qiskit_full, qiskit_linear

from PatchedMeasCal import non_standard_coupling_maps


from functools import partial

import qiskit.tools.jupyter

import mthree

In [25]:
corr_err_str = [100, 0, 10] + [0] * (n_qubits - 2)
probs = FakeMeasurementError(
    corr_err_str, # Const - Controls correlation of error weights
    [0] * (n_qubits + 1), # 1 -> 0 - Controls error biases
    [0] * (n_qubits + 1), # 0 -> 1 - Controls error biases
    n_qubits = n_qubits, 
    coupling_map = backend._coupling_map)

In [26]:
n_qubits = 10

In [37]:
n_shots = 8000
n_rounds = 5

circuit_res = []
aim_res = []
sim_res = []
full_res = []
lin_res = []
tpf_res = []
tpf_e_res = []
jr_res = []
m3_res = []

backend_generators = [
    partial(Hexagonal, 1, 1),
    #partial(Hexagonal, 2, 1),
    #Hexagonal13,
    #partial(Hexagonal, 3, 1),
    #Hexagonal16
]

circuit_generators = [state_prep_circuits.plus_state_prep]

pb = Progressbar(20, 5 * n_rounds * len(backend_generators), 'Sims')

errs = list(range(0, 30, 5))

for err in errs:
    for backend_gen in backend_generators:
    
        circuit_round = []
        aim_round = []
        sim_round = []
        tpf_round = []
        tpf_e_round = []
        jr_round = []
        m3_round = []
        full_round = []
        linear_round = []

        for r_num in range(n_rounds):


            #backend = FullyConnected(n_qubits)
            backend = backend_gen()
            n_qubits = len(backend.properties().qubits)
            err_cmap = non_standard_coupling_maps.error_coupling_map(backend=backend)

            corr_err_str = [100, 0, err] + [0] * (n_qubits - 2)
            probs = FakeMeasurementError(
                corr_err_str, # Const - Controls correlation of error weights
                [0] * (n_qubits + 1), # 1 -> 0 - Controls error biases
                [0] * (n_qubits + 1), # 0 -> 1 - Controls error biases
                n_qubits = n_qubits, 
                coupling_map = backend._coupling_map)

            # Build tensor patches
            tpf = TensorPatchFitter(backend, n_shots=n_shots / 2)
            tpf.build(probs=probs)

            tpf_e = TensorPatchFitter(backend, n_shots=n_shots / 2, coupling_map=err_cmap)
            tpf_e.build(probs=probs)

            full = qiskit_full(backend, n_qubits, n_shots, probs=probs)
            linear = qiskit_linear(backend, n_qubits, n_shots, probs=probs)

            # Run Circuits

            circuit = state_prep_circuits.plus_state_prep(backend)      
            pb.tick('Bare: {}'.format(n_qubits))
            # Transpile and run circuit
            tc = qiskit.transpile(circuit,
                                  backend,
                                  optimization_level=0,
                                  initial_layout=list(range(n_qubits))
                                 )
            cr = qiskit.execute(circuit, 
                                 backend, 
                                 shots=n_shots / 2, 
                                 optimization_level=0,
                                 initial_layout=list(range(n_qubits))
                                ).result().get_counts()
            cr = probs(cr)
            circuit_round.append(state_prep_circuits.plus_state_dist(cr))
            

            # Apply AIM to circuit
            pb.tick('AIM {}'.format(n_qubits))
            ar = aim(circuit, backend, n_qubits, n_shots=n_shots, equal_shot_distribution=True, probs=probs)
            aim_round.append(state_prep_circuits.plus_state_dist(ar))

            # Apply SIM to circuit
            pb.tick('SIM {}'.format(n_qubits))
            sr = sim(circuit, backend, n_qubits, n_shots=n_shots, equal_shot_distribution=True, probs=probs) 
            sr = renormalise_measurement_results(sr, 1)
            sim_round.append(state_prep_circuits.plus_state_dist(sr))

            # Apply Full
            pb.tick('Full {}'.format(n_qubits))
            fr = full.apply(cr)
            full_round.append(state_prep_circuits.plus_state_dist(fr))

            # Apply Linear
            pb.tick('Lin {}'.format(n_qubits))
            lr = linear.apply(cr)
            linear_round.append(state_prep_circuits.plus_state_dist(lr))

            # Apply CMC to circuit
            pb.tick('CMC {}'.format(n_qubits))
            tr = tpf.apply(cr)
            tpf_round.append(state_prep_circuits.plus_state_dist(tr))

            # Apply CMC to circuit
            pb.tick('CMC E{}'.format(n_qubits))
            tr = tpf_e.apply(cr)
            tpf_e_round.append(state_prep_circuits.plus_state_dist(tr))

            # Jigsaw
            pb.tick('JIG {}'.format(n_qubits))
            jr = jigsaw.jigsaw(circuit, backend, n_shots, equal_shot_distribution=True, probs=probs)
            jr_round.append(state_prep_circuits.plus_state_dist(jr))

             # M3
            mit = mthree.M3Mitigation(backend, probs=probs)
            mit.cals_from_system(list(range(n_qubits)), n_shots // 2)
            m3_round.append(state_prep_circuits.plus_state_dist(
                mit.apply_correction(cr, list(range(n_qubits))).nearest_probability_distribution()
            ))


        circuit_res.append(circuit_round)
        aim_res.append(aim_round)
        sim_res.append(sim_round)
        full_res.append(full_round)
        lin_res.append(linear_round)
        tpf_res.append(tpf_round)
        tpf_e_res.append(tpf_e_round)
        jr_res.append(jr_round)
        m3_res.append(m3_round)

results = {
'bare':circuit_res,
'aim':aim_res,
'sim':sim_res,
'full':full_res,
'lin':lin_res,
'cmc':tpf_res,
'cmc_err':tpf_e_res,
'jig':jr_res,
'm3':m3_res
}

Sims : [====================] 100.0% Bare: 6 ETA: 0s
Sims : [====================] 100.0% AIM 6 ETA: 0s
Sims : [====================] 100.0% SIM 6 ETA: 0s
Sims : [====================] 100.0% Full 6 ETA: 0s
Sims : [====================] 100.0% Lin 6 ETA: 0s
Sims : [====================] 100.0% CMC 6 ETA: 0s
Sims : [====================] 100.0% CMC E6 ETA: 0s
Sims : [====================] 100.0% JIG 6 ETA: 0s
Sims : [====================] 100.0% Bare: 6 ETA: 0s
Sims : [====================] 100.0% AIM 6 ETA: 0s
Sims : [====================] 100.0% SIM 6 ETA: 0s
Sims : [====================] 100.0% Full 6 ETA: 0s
Sims : [====================] 100.0% Lin 6 ETA: 0s
Sims : [====================] 100.0% CMC 6 ETA: 0s
Sims : [====================] 100.0% CMC E6 ETA: 0s
Sims : [====================] 100.0% JIG 6 ETA: 0s
Sims : [====================] 100.0% Bare: 6 ETA: 0s
Sims : [====================] 100.0% AIM 6 ETA: 0s
Sims : [====================] 100.0% SIM 6 ETA: 0s
Sims : [=============

Sims : [====================] 100.0% JIG 6 ETA: 0s
Sims : [====================] 100.0% Bare: 6 ETA: 0s
Sims : [====================] 100.0% AIM 6 ETA: 0s
Sims : [====================] 100.0% SIM 6 ETA: 0s
Sims : [====================] 100.0% Full 6 ETA: 0s
Sims : [====================] 100.0% Lin 6 ETA: 0s
Sims : [====================] 100.0% CMC 6 ETA: 0s
Sims : [====================] 100.0% CMC E6 ETA: 0s
Sims : [====================] 100.0% JIG 6 ETA: 0s
Sims : [====================] 100.0% Bare: 6 ETA: 0s
Sims : [====================] 100.0% AIM 6 ETA: 0s
Sims : [====================] 100.0% SIM 6 ETA: 0s
Sims : [====================] 100.0% Full 6 ETA: 0s
Sims : [====================] 100.0% Lin 6 ETA: 0s
Sims : [====================] 100.0% CMC 6 ETA: 0s
Sims : [====================] 100.0% CMC E6 ETA: 0s
Sims : [====================] 100.0% JIG 6 ETA: 0s
Sims : [====================] 100.0% Bare: 6 ETA: 0s
Sims : [====================] 100.0% AIM 6 ETA: 0s
Sims : [=============

In [36]:
results

{'bare': [[0.30950000000000005, 0.34575, 0.2605, 0.3035, 0.2855],
  [0.35675,
   0.32725,
   0.31575000000000003,
   0.31575000000000003,
   0.31224999999999997],
  [0.32725000000000004,
   0.25475000000000003,
   0.318,
   0.287,
   0.31525000000000003],
  [0.33175, 0.28875, 0.30725, 0.31525000000000003, 0.29275],
  [0.25125, 0.28625, 0.30600000000000005, 0.3535, 0.31175],
  [0.29700000000000004, 0.25575, 0.2925, 0.308, 0.30250000000000005],
  [0.28250000000000003, 0.289, 0.319, 0.30924999999999997, 0.3465],
  [0.28125, 0.27025, 0.27425, 0.32875000000000004, 0.29274999999999995],
  [0.3125, 0.30575, 0.297, 0.31124999999999997, 0.30525],
  [0.30725, 0.2995, 0.28350000000000003, 0.26999999999999996, 0.2835],
  [0.295, 0.344, 0.29550000000000004, 0.3275, 0.30000000000000004],
  [0.32625, 0.2965, 0.30025, 0.31074999999999997, 0.34675],
  [0.24774999999999997, 0.28625, 0.25475000000000003, 0.2935, 0.31175],
  [0.27475000000000005, 0.307, 0.27625, 0.29974999999999996, 0.26725],
  [0.3192500

In [33]:
results

{'bare': [[0.32499999999999996,
   0.3115,
   0.26625,
   0.30774999999999997,
   0.29700000000000004]],
 'aim': [[0.328,
   0.286,
   0.2781666666666667,
   0.3198333333333333,
   0.29783333333333334]],
 'sim': [[0.32574999787247005,
   0.3199999970651699,
   0.26800000487751074,
   0.32124999631427287,
   0.30087499532052997]],
 'full': [[0.05508867591651828,
   0.05872552443042073,
   0.0466781475159499,
   0.03687028806919207,
   0.043548056204455454]],
 'lin': [[0.061883644539412974,
   0.06351421749974495,
   0.05301076410401201,
   0.03588840434701368,
   0.044163626447260795]],
 'cmc': [[0.10309054654375843,
   0.15184567690387157,
   0.088662045608631,
   0.12184214063397042,
   0.07170635717752888]],
 'cmc_err': [[0.10427544290500657,
   0.19535663957636706,
   0.09709624098205205,
   0.14583177937877084,
   0.0753301490549042]],
 'jig': [[0.20851227756864676,
   0.23650544639970017,
   0.16330341636450457,
   0.20660218364796779,
   0.18070699804480606]],
 'm3': [[0.06179754

In [10]:
results

{'bare': [[0.26975000000000005,
   0.30424999999999996,
   0.28375,
   0.30175,
   0.28150000000000003],
  [0.40075, 0.45275, 0.4625, 0.484, 0.455],
  [0.5745, 0.5269999999999999, 0.5605, 0.54225, 0.509]],
 'aim': [[0.263,
   0.29883333333333334,
   0.30516666666666664,
   0.30316666666666664,
   0.2871666666666667],
  [0.408,
   0.46149999999999997,
   0.46849999999999997,
   0.47783333333333333,
   0.4515],
  [0.5841666666666666, 0.5455000000000001, 0.5585, 0.5375, 0.522]],
 'sim': [[0.2748749965261525,
   0.29662500734519726,
   0.2918750002122579,
   0.29299999635516744,
   0.2834999956367904],
  [0.41475000170696874,
   0.4511249972386633,
   0.46874999475812756,
   0.4827500103492091,
   0.45399999708379635],
  [0.5765000022982785,
   0.5482500052030699,
   0.5662500012661621,
   0.5402499964228593,
   0.515624996030738]],
 'full': [],
 'lin': [],
 'cmc': [[0.11941521080195494,
   0.11408478794031046,
   0.108406226345576,
   0.10654230014870453,
   0.10541099094609657],
  [0.145

In [25]:
results

{'bare': [[0.28025, 0.34225, 0.3065, 0.30825, 0.26575],
  [0.35350000000000004, 0.374, 0.36025, 0.34525, 0.36324999999999996],
  [0.381, 0.38375, 0.39325, 0.383, 0.3865],
  [0.40075,
   0.43575,
   0.40349999999999997,
   0.39849999999999997,
   0.45325000000000004],
  [0.46075, 0.40750000000000003, 0.44975, 0.45449999999999996, 0.45225],
  [0.455, 0.45125, 0.5065, 0.45675, 0.50875],
  [0.5105000000000001, 0.51025, 0.504, 0.54775, 0.505],
  [0.53, 0.50825, 0.56525, 0.5345, 0.50225],
  [0.55175, 0.5705, 0.565, 0.5175000000000001, 0.60775],
  [0.60625, 0.5872499999999999, 0.6174999999999999, 0.6135, 0.6025],
  [0.5972500000000001, 0.6435, 0.6587500000000001, 0.64025, 0.613]],
 'aim': [],
 'sim': [],
 'full': [],
 'lin': [],
 'cmc': [[0.18763105971810023,
   0.12112439929353602,
   0.3240318159234237,
   0.15460440959708527,
   0.16649727260555858],
  [0.1477494734886356,
   0.270292349779756,
   0.11519895518674922,
   0.10631119028766345,
   0.2647008599814858],
  [0.36379795022619144,


[[0.1258825408085893,
  0.05456219094257392,
  0.07325000322209063,
  0.0966495634587734,
  0.1677002952465544],
 [0.1258825408085893,
  0.05456219094257392,
  0.07325000322209063,
  0.0966495634587734,
  0.1677002952465544],
 [0.1258825408085893,
  0.05456219094257392,
  0.07325000322209063,
  0.0966495634587734,
  0.1677002952465544],
 [0.1258825408085893,
  0.05456219094257392,
  0.07325000322209063,
  0.0966495634587734,
  0.1677002952465544],
 [0.1258825408085893,
  0.05456219094257392,
  0.07325000322209063,
  0.0966495634587734,
  0.1677002952465544]]

In [10]:
res_6 = {'bare': [[0.28825, 0.279, 0.26449999999999996, 0.27525, 0.305]],
 'aim': [[0.29583333333333334,
   0.2791666666666667,
   0.2796666666666667,
   0.2763333333333333,
   0.29766666666666663]],
 'sim': [[0.2888749947365471,
   0.27625000432912183,
   0.2830000079168529,
   0.28674999778928045,
   0.2973749998018775]],
 'full': [[0.04253703767664013,
   0.04917375679451208,
   0.02475919484672745,
   0.0344105440503269,
   0.06443661920216953]],
 'lin': [[0.04959991156610394,
   0.04468267154149502,
   0.024604316464146625,
   0.0435101165457884,
   0.06547682885391809]],
 'cmc': [[0.09000196101075975,
   0.08839767732893877,
   0.08372137586198558,
   0.08720267506320495,
   0.09686944927298952]],
 'cmc_err': [[0.07505142178099061,
   0.16827448829090946,
   0.0680343885354846,
   0.1005783632142645,
   0.09382542144684536]],
 'jig': [[0.21624954352304493,
   0.2299136648888655,
   0.18030373596506477,
   0.1848477844392224,
   0.21544202044782612]]}
res_10 = {'bare': [[0.44975, 0.48425, 0.43274999999999997, 0.407, 0.46]],
 'aim': [[0.45316666666666666,
   0.49216666666666664,
   0.4421666666666667,
   0.40399999999999997,
   0.477]],
 'sim': [[0.45475001034087814,
   0.4938749964039218,
   0.45637499883203786,
   0.4082500010226903,
   0.470374997069188]],
 'full': [],
 'lin': [],
 'cmc': [[0.18547797376058017,
   0.19909542105639827,
   0.18272212481574007,
   0.17624603281190426,
   0.20193650461971208]],
 'cmc_err':[[0.180740800256449,
   0.22876394236328312,
   0.19326759752528394,
   0.18415668218067754,
   0.1326737622418932]],
 'jig': [[0.3364957043132925,
   0.32200201984181215,
   0.28965856044363136,
   0.23635762871954558,
   0.3725673359792167]]}
res_13 = {'bare': [[0.51825, 0.5782499999999999, 0.5415, 0.556, 0.5900000000000001]],
 'aim': [[0.5089999999999999,
   0.5753333333333334,
   0.5555,
   0.554,
   0.5916666666666667]],
 'sim': [[0.519624999904172,
   0.571249998803287,
   0.550499997528532,
   0.5511250043241397,
   0.5852499969364692]],
 'full': [],
 'lin': [],
 'cmc': [[0.22927965652962679,
   0.2977425623080594,
   0.25154726688165135,
   0.2733705001180055,
   0.29660532541206436]],
 'cmc_err': [[0.2954222766363798,
   0.25882904748285956,
   0.2581545763119574,
   0.21804752045956916,
   0.3198949620065737]],
 'jig': [[0.37940010935237645,
   0.4427584288510964,
   0.44571389520492016,
   0.4291114378630857,
   0.459763787370337]]}
res_14 = {'bare': [[0.581, 0.57975, 0.54225, 0.56525, 0.554]],
 'aim': [[0.5791666666666666,
   0.5868333333333333,
   0.5438333333333334,
   0.5655,
   0.5476666666666666]],
 'sim': [[0.5772499946148276,
   0.5918749953260888,
   0.553125000012642,
   0.5688750011406664,
   0.5551250027358418]],
 'full': [],
 'lin': [],
 'cmc': [[0.3284713334930113,
   0.2932916434089294,
   0.30636594448630344,
   0.279176738455104,
   0.3410652090519649]],
 'cmc_err': [[0.25634236166658014,
   0.32455338047684157,
   0.345039109124481,
   0.3530454031665931,
   0.30210461216703577]],
 'jig': [[0.4626437518191554,
   0.4618972206480476,
   0.3833361955803623,
   0.4258315959786799,
   0.41035520258673736]]}
res_16 = {'bare': [[0.6665000000000001,
   0.63375,
   0.6072500000000001,
   0.6725,
   0.6227499999999999]],
 'aim': [[0.6558333333333333,
   0.6236666666666667,
   0.612,
   0.6566666666666667,
   0.624]],
 'sim': [[0.6662500024010296,
   0.6126250031798917,
   0.6206249941614718,
   0.6607499909915282,
   0.6298749936203825]],
 'full': [],
 'lin': [],
 'cmc': [[0.41670307422429537,
   0.4099541436248002,
   0.34718461110838655,
   0.42844927394227683,
   0.34615592816670643]],
 'cmc_err': [[0.35759553472207495,
   0.3358319112003164,
   0.35371629204840793,
   0.3607940354634598,
   0.36564654699523974]],
 'jig': [[0.5671885255957472,
   0.4920644730921216,
   0.46824052999432325,
   0.4890236048334911,
   0.46692541877604576]]}



In [21]:
range_results_cmc_conv = {'bare': [[0.22800000000000004,
   0.276,
   0.22649999999999998,
   0.24625000000000002,
   0.26599999999999996],
  [0.30375, 0.33675, 0.30875, 0.332, 0.33225],
  [0.38175, 0.33425000000000005, 0.35, 0.30975, 0.36974999999999997],
  [0.39299999999999996,
   0.35374999999999995,
   0.402,
   0.3705,
   0.37825000000000003],
  [0.43, 0.41325, 0.42075, 0.45075, 0.39475],
  [0.5117499999999999,
   0.432,
   0.42400000000000004,
   0.4745,
   0.43674999999999997],
  [0.46575, 0.495, 0.49074999999999996, 0.462, 0.46275],
  [0.522, 0.50025, 0.5015000000000001, 0.49725, 0.49424999999999997],
  [0.5395, 0.5395000000000001, 0.5377500000000001, 0.52525, 0.533],
  [0.55375, 0.5547500000000001, 0.5645, 0.5660000000000001, 0.56525],
  [0.5920000000000001, 0.58375, 0.5974999999999999, 0.59425, 0.60375],
  [0.6252500000000001, 0.61825, 0.6165, 0.6155, 0.64725]],
 'aim': [],
 'sim': [],
 'full': [],
 'lin': [],
 'cmc': [[0.05358225009386208,
   0.25429361389550936,
   0.34021763745659556,
   0.2797774360388859,
   0.07673402877032487],
  [0.13942624509020352,
   0.09993643325706647,
   0.11786690295919888,
   0.10210052392676389,
   0.1406134746515798],
  [0.13121877653455594,
   0.11152721435917418,
   0.40687007793114405,
   0.20262232205279346,
   0.18227712564542703],
  [0.141244595797826,
   0.255235047736292,
   0.14693579082143643,
   0.4809055461030687,
   0.3483854682269039],
  [0.24238155859315236,
   0.13971478251212488,
   0.5104084374122126,
   0.22947809129985652,
   0.1573919958168593],
  [0.5523038931435346,
   0.15988157359831212,
   0.13433522122397562,
   0.275967438268855,
   0.17556336854847826],
  [0.18835336135236502,
   0.5944753899982071,
   0.2391338303376886,
   0.34968506806933775,
   0.64981874671702],
  [0.5682083045398937,
   0.23352151283491407,
   0.22342151718396336,
   0.23424771719885906,
   0.5698882454635708],
  [0.5313459556961398,
   0.6706903282408925,
   0.43101168257193534,
   0.29227203794641776,
   0.67571299762956],
  [0.26349422514953774,
   0.6788406971085215,
   0.47946769009227286,
   0.5376713904124062,
   0.29524281710190636],
  [0.39745242069263864,
   0.6752266866444963,
   0.38163844599658686,
   0.7169692360266309,
   0.8031060320089987],
  [0.36308213219985025,
   0.6071966270170672,
   0.36300667349218035,
   0.3661786920732499,
   0.5935334400211769]],
 'jig': []}

In [26]:
range_results_cmc_2 = {'bare': [[0.28025, 0.34225, 0.3065, 0.30825, 0.26575],
  [0.35350000000000004, 0.374, 0.36025, 0.34525, 0.36324999999999996],
  [0.381, 0.38375, 0.39325, 0.383, 0.3865],
  [0.40075,
   0.43575,
   0.40349999999999997,
   0.39849999999999997,
   0.45325000000000004],
  [0.46075, 0.40750000000000003, 0.44975, 0.45449999999999996, 0.45225],
  [0.455, 0.45125, 0.5065, 0.45675, 0.50875],
  [0.5105000000000001, 0.51025, 0.504, 0.54775, 0.505],
  [0.53, 0.50825, 0.56525, 0.5345, 0.50225],
  [0.55175, 0.5705, 0.565, 0.5175000000000001, 0.60775],
  [0.60625, 0.5872499999999999, 0.6174999999999999, 0.6135, 0.6025],
  [0.5972500000000001, 0.6435, 0.6587500000000001, 0.64025, 0.613]],
 'aim': [],
 'sim': [],
 'full': [],
 'lin': [],
 'cmc': [[0.18763105971810023,
   0.12112439929353602,
   0.3240318159234237,
   0.15460440959708527,
   0.16649727260555858],
  [0.1477494734886356,
   0.270292349779756,
   0.11519895518674922,
   0.10631119028766345,
   0.2647008599814858],
  [0.36379795022619144,
   0.386838965271883,
   0.351206183334842,
   0.2902114664361831,
   0.6513463405975657],
  [0.23840574420698873,
   0.18812375191299535,
   0.1984002423542729,
   0.1386609168737012,
   0.5798638083608105],
  [0.18039660691444376,
   0.7580900958733008,
   0.35357860752469134,
   0.17698938332423336,
   0.3932398226223104],
  [0.7280257389316738,
   0.746398797769257,
   0.7398654121309339,
   0.19260973960215672,
   0.36004072045732155],
  [0.5490760716600565,
   0.44214936560757734,
   0.22142876954071744,
   0.6740539855379124,
   0.24229254858068278],
  [0.547753484476457,
   0.6992701038854923,
   0.562822633279497,
   0.6712412777258755,
   0.6996210695712825],
  [0.2726694200515931,
   0.2884052603978325,
   0.28721112970903434,
   0.4534390290690393,
   0.2946438253479356],
  [0.4440217939387644,
   0.742759884417757,
   0.7660011487198338,
   0.758941467171486,
   0.6596463964351803],
  [0.3303704632299451,
   0.4875838582557468,
   0.3719403449681935,
   0.37042841982834873,
   0.33703068722636054]],
 'jig': []}

In [ ]:
grid_8 = {
    'bare': [[0.39325, 0.384, 0.3585, 0.40075, 0.39775]],
 'aim': [[0.3995, 0.37733333333333335, 0.365, 0.4055, 0.39949999999999997]],
 'sim': [[0.39099999914962974,
   0.3793749932551241,
   0.37962499806868083,
   0.3996249988629589,
   0.3997499982181144]],
 'full': [[0.07492761054176378,
   0.08232543446166041,
   0.0376145204554546,
   0.058486103762973585,
   0.07866355920837509]],
 'lin': [[0.0705519444588289,
   0.08036422117125419,
   0.0485408473537029,
   0.06326659044990163,
   0.07196727696074628]],
 'cmc': [[0.15660892603997778,
   0.14115815072539106,
   0.11383491002329238,
   0.1449464955327902,
   0.16516712757807706]],
 'jig': [[0.3160183670094996,
   0.23250761964867084,
   0.30039176829470066,
   0.26167304349961473,
   0.26468659481447854]]}